In [58]:
import pandas as pd
import os

# Path relative to the script's directory
script_dir = os.getcwd()
print (script_dir)
path = os.path.join(script_dir, "../results/data_formatted_subjectAreas.csv")

df = pd.read_csv(path)
df["Processed Words"]

c:\Users\PangSunatcha\OneDrive - Chulalongkorn University\Documents\Y2S1 files\Data Sci\proj\Data-Sci-project\src


0        ['recent', 'developments', 'bifunctional', 'ai...
1        ['benefit', 'punishment', 'sensitivity', 'moto...
2        ['magic', 'biomarkers', 'predict', 'longterm',...
3        ['endocrine', 'disrupting', 'compounds', 'envi...
4        ['probabilistic', 'regular', 'grammar', 'infer...
                               ...                        
20211    ['prediction', 'diagnosis', 'chronic', 'kidney...
20212    ['incidence', 'healthcareassociated', 'urinary...
20213    ['turmericloaded', 'alginate', 'particulatebas...
20214    ['identification', 'novel', 'gut', 'microbiota...
20215    ['effectiveness', '1', '2per', 'cent', 'acetic...
Name: Processed Words, Length: 20216, dtype: object

In [51]:
import ast

df['Processed Words'] = df['Processed Words'].apply(ast.literal_eval)
df['Author Keywords'] = df['Author Keywords'].fillna('')
df['combined keywords'] = df.apply(
    lambda row: row['Processed Words'] + row['Author Keywords'].split(', '),
    axis=1
)
df['combined keywords'].apply(lambda row: list(set(row)))

0        [fuel, cells, Bifunctional air electrode, Cata...
1        [benefit, performance, punishment, reinforceme...
2        [, biomarkers, gvhd, predict, acute, outcomes,...
3        [Epigenetics, endocrine, Gene expression, Endo...
4        [Probabilistic finite state machine, inference...
                               ...                        
20211    [metaanalysis, development, , chronic, systema...
20212    [infections, tertiary care hospital, child, tr...
20213    [Gas-generating agent, release, Turmeric extra...
20214    [, cancer, signature, identification, gut, nov...
20215    [cent, Tympanic membrane, acetic, granular, 1,...
Name: combined keywords, Length: 20216, dtype: object

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Prepare features and labels
# TF-IDF feature extraction
df['combined keywords'] = df['combined keywords'].apply(lambda x: ' '.join(x))
tfidf = TfidfVectorizer(max_features=5000)
tfidf_kw = tfidf.fit_transform(df['combined keywords'])

new_tfidf = TfidfVectorizer(max_features=5000)
tfidf_pn = new_tfidf.fit_transform(df['Publication Name'])

from scipy.sparse import hstack
X = hstack([tfidf_kw, tfidf_pn])
y = df['Subject Areas']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


# Train Logistic Regression model
logreg = LogisticRegression(C = 100, max_iter= 500, penalty= 'l2', solver = 'saga' ,random_state=50) 
logreg.fit(X_train, y_train)

# Evaluate the model
y_pred = logreg.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8936696340257171

Classification Report:
                                       precision    recall  f1-score   support

Agricultural and Biological Sciences       0.93      0.90      0.91       220
                 Arts and Humanities       0.97      0.88      0.92        33
                        Biochemistry       0.84      0.78      0.81       236
                            Business       0.82      0.85      0.83        65
                Chemical Engineering       0.94      0.82      0.88       102
                           Chemistry       0.87      0.96      0.92       260
                    Computer Science       0.87      0.90      0.88       232
                   Decision Sciences       1.00      0.20      0.33         5
                           Dentistry       0.97      0.90      0.93        70
        Earth and Planetary Sciences       0.88      0.79      0.84        58
            Econometrics and Finance       0.80      0.77      0.79        31
         

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Load the dataset
# df = pd.read_csv("/path/to/data_formatted_subjectAreas.csv")
# df['Processed Words'] = df['Processed Words'].apply(eval)

# # Prepare features and labels
# X = df['Processed Words'].apply(lambda x: ' '.join(x))
# y = df['Subject Areas']

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # TF-IDF feature extraction
# tfidf = TfidfVectorizer(max_features=5000)
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [100, 500, 1000]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=50),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Train the model
grid_search.fit(X_train, y_train)

# Best parameters and evaluation
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\PangSunatcha\.conda\envs\DSenv\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Parameters: {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}
Accuracy: 0.8919386745796242

Classification Report:
                                       precision    recall  f1-score   support

Agricultural and Biological Sciences       0.93      0.90      0.91       220
                 Arts and Humanities       0.97      0.88      0.92        33
                        Biochemistry       0.83      0.78      0.81       236
                            Business       0.82      0.85      0.83        65
                Chemical Engineering       0.93      0.82      0.88       102
                           Chemistry       0.87      0.96      0.91       260
                    Computer Science       0.87      0.89      0.88       232
                   Decision Sciences       1.00      0.20      0.33         5
                           Dentistry       0.97      0.91      0.94        70
        Earth and Planetary Sciences       0.89      0.81      0.85        58
       